In [3]:
import numpy as np
import pandas as pd

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

### Đọc dữ liệu là tập dataset IRIS ###

In [5]:
dataset = pd.read_csv("../data/IRIS.csv")
dataset.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Chia tập dữ liệu ra thành 2 tập gồm tập đặc trưng và tập nhãn tương ứng ###

In [6]:
X = np.array(dataset.iloc[:, :-1], dtype=float)
y = np.array(dataset.iloc[:, -1])

### Chia tập dữ liệu ra thành tập train và tập test ###

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
X_train.shape

(105, 4)

### Xây dựng class GaussianBY dựa trên Gaussian Naive Bayes để phân lớp ###

In [9]:
class GaussianBY():
  def separate_by_classes(self, X, y):
    # Chia tập dữ liệu tương ứng với lớp của nó
    self.classes = np.unique(y)
    classes_id = {}
    subdatasets = {}
    cls, counts = np.unique(y, return_counts=True)
    self.class_freq = dict(zip(cls, counts))
    self.class_freq_real = dict(zip(cls, counts))
    N = sum(list(self.class_freq.values()))
    for class_type in self.classes:
      classes_id[class_type] = np.argwhere(y==class_type).reshape(1, -1)[0]
      subdatasets[class_type] = X[classes_id[class_type]]
      self.class_freq[class_type] = self.class_freq[class_type] / N
    # Kết quả trẻ về là dict với các khóa là tên lớp và value là tập dữ liệu thuộc lớp đó
    return subdatasets
  def fit(self, X, y):
    # Tính mean và ma trận hiệp phương sai của từng lớp
    separate_X = self.separate_by_classes(X, y)
    self.means = {}
    self.covs = {}
    for class_type in self.classes:
      # Tính mean
      self.means[class_type] = np.mean(separate_X[class_type], axis=0)
      # Tính ma trận hiệp phương sai
      #self.covs[class_type] = np.cov(separate_X[class_type].T)
      self.covs[class_type] = ((separate_X[class_type] - self.means[class_type]).T.dot((separate_X[class_type] - self.means[class_type]))) / self.class_freq_real[class_type]
  def pdf(self, x, mean, cov):
    # Tính mật độ xác suất tại x của một phân phổi chuẩn có kỳ vọng là mean
    # và ma trận hiệp phương sai cov
    numberator = np.exp(np.dot(np.dot((x - mean).T, (np.linalg.inv(cov))), (x - mean)) * -1/2)
    denominator = np.power((2*np.pi), x.size/2) * np.power((np.linalg.det(cov)), 1/2)
    return numberator / denominator
  def predict_prob(self, X):
    # Tính xác suất của dữ liệu X tương ứng với từng lớp
    px_class = {}
    self.P_classes = {}
    for class_type in self.classes:
      px_class[class_type] = self.pdf(X, self.means[class_type], self.covs[class_type])
      self.P_classes[class_type] = (px_class[class_type] * self.class_freq[class_type])
    for class_type in self.classes:
      self.P_classes[class_type] = self.P_classes[class_type] / sum(list(px_class.values()))
    # Trả về một dict với key là tên lớp và value là xác suất của dữ liệu x thuộc về lớp đó
    return self.P_classes
  def predict(self, X):
    # Dự đoán x thuộc về lớp nào dựa trên việc chọn xác suất lớn nhất của x đối với các lớp
    pred = []
    for x in X:
      pred_class = None
      max_prob = 0
      for cls, prob in self.predict_prob(x).items():
        if prob > max_prob:
          max_prob = prob
          pred_class = cls
      pred.append(pred_class)
    return pred

### Phân lớp Iris dựa trên class GaussianBY đã xây dựng ###

In [10]:
model = GaussianBY()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred)
print(f"Kết quả dự đoán dựa trên class GaussianBY đã xây dựng có độ chính xác là: {score*100}%")

Kết quả dự đoán dựa trên class GaussianBY đã xây dựng có độ chính xác là: 97.77777777777777%


### Phân lớp Iris dựa trên GaussianNB của sklearn ###

In [16]:
from sklearn.naive_bayes import GaussianNB
gau = GaussianNB()
gau.fit(X_train, y_train)
gau_pred = gau.predict(X_test)
score_2 = accuracy_score(y_test, gau_pred)
print(f"Kết quả dự đoán dựa trên GaussianNB của sklearn có độ chính xác là: {score_2*100}%")

Kết quả dự đoán dựa trên GaussianNB của sklearn có độ chính xác là: 100.0%


### Phân lớp Iris dựa trên MultinomialNB của sklearn ###

In [13]:
from sklearn.naive_bayes import MultinomialNB
mul = MultinomialNB()
mul.fit(X_train, y_train)
mul_pred = mul.predict(X_test)
score_3 = accuracy_score(y_test, mul_pred)
print(f"Kết quả dự đoán dựa trên MultinomialNB của sklearn có độ chính xác là: {score_3*100}%")

Kết quả dự đoán dựa trên MultinomialNB của sklearn có độ chính xác là: 60.0%


### Phân lớp Iris dựa trên BeNB của sklearn ###

In [14]:
from sklearn.naive_bayes import BernoulliNB
ber = BernoulliNB()
ber.fit(X_train, y_train)
ber_pred = ber.predict(X_test)
score_4 = accuracy_score(y_test, ber_pred)
print(f"Kết quả dự đoán dựa trên BernoulliNB của sklearn có độ chính xác là: {score_3*100}%")

Kết quả dự đoán dựa trên BernoulliNB của sklearn có độ chính xác là: 60.0%
